In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np


import pandas as pd
import time
import random


from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()


frescos = {
    "Frutas": "https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=",
    "Carne": "https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=",
    "Panadería":"https://www.alcampo.es/compra-online/frescos/panaderia/c/W1281?q=%3Arelevance&page=",
    "Verduras y hortalizas":"https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/c/W1702?q=%3Arelevance&page=",
    "Charcutería":"https://www.alcampo.es/compra-online/frescos/charcuteria/c/W15?q=%3Arelevance&page=",
    "Pastelería":"https://www.alcampo.es/compra-online/frescos/pasteleria/c/W1282?q=%3Arelevance&page=",
    "Sushi": "https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=",
    "Quesos": "https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=",
    "Pescados, mariscos y moluscos": "https://www.alcampo.es/compra-online/frescos/pescados-mariscos-y-moluscos/c/W14?q=%3Arelevance&page=",
    "Ahumados, sucedáneos gulas, anchoas": "https://www.alcampo.es/compra-online/frescos/ahumados-sucedaneos-gulas-anchoas/c/W184?q=%3Arelevance&page=",
    "Fresco al Corte": "https://www.alcampo.es/compra-online/frescos/fresco-al-corte/c/WFrescoalcorte?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in frescos.items():

    categoria = i[0]
    url_to_scrape = i[1]
    print(categoria,url_to_scrape)

    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")

        # print(soup.find("div",class_="totalResults").get_text(strip=True, separator=''))
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:

            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")


            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )

        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo


Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Category: 186 Name: 186 Image: 186 Price: 186 Stock 186
No hay más ventanas
Carne https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=
Category: 234 Name: 234 Image: 234 Price: 234 Stock 234
Category: 282 Name: 282 Image: 282 Price: 282 Stock 282
Category: 330 Name: 330 Image: 330 Price: 330 Stock 330
Category: 378 Name: 378 Image: 378 Price: 378 Stock 378
Category: 426 Name: 426 Image: 426 Price: 426 Stock 426
Category: 474 Name: 474 Image: 474 Price: 474 Stock 474
Category: 522 Name: 522 Image: 522 Price: 522 Stock 522
Category: 570 Name: 570 Image: 570 Price: 570 Stock 570
Category: 618 Name: 618 Image: 618 Price: 618 Stock 618
Category: 666 Name: 666 Image: 666 Price: 666 Stock 666
Category: 714 Name: 714 Image: 714

,Category,Name,Price,Stock,Image
0,Frutas,Fresón Leyre ALCAMPO PRODUCCIÓN CONTROLADA 400 g.,"(7,47 €/Kg)",Yes,https://www.alcampo.es/media/hc7/hd3/978298965...
1,Frutas,Plátano de Canarias (Indicación Geográfica Pro...,"(3,55 €/Kg)",Yes,https://www.alcampo.es/media/h82/hd5/930404787...
2,Frutas,Plátano Canario extra bolsa,"1,89 €(1,89 €/Kg)",Yes,https://www.alcampo.es/media/h7e/ha2/111832744...
3,Frutas,Banana a granel,"1,59 €(1,59 €/Kg)",Yes,https://www.alcampo.es/media/h3c/h6c/118745949...
4,Frutas,Aguacate malla de 500 g.,"1,79 €(3,58 €/Kg)",Yes,https://www.alcampo.es/media/hee/he4/980467312...
...,...,...,...,...,...
4319,Fresco al Corte,Costillar de bellota ibérico de cerdo ALCAMPO ...,"2,24 €(8,95 €/Kg)",Yes,https://www.alcampo.es/media/h83/h03/109049182...
4320,Fresco al Corte,Chicharro mostrador,"5€(9,99 €/Kg)",Yes,https://www.alcampo.es/media/h55/hb1/110263468...
4321,Fresco al Corte,Rape cola pequeña mostrador,"16,07 €(22,95 €/Kg)",Yes,https://www.alcampo.es/media/h80/h75/111321900...
4322,Fresco al Corte,"Medio cordero lechal, especial para asados y c...","69,83 €(19,95 €/Kg)",No,https://www.alcampo.es/media/h33/h0e/109069259...
